In [1]:
# %load movie_review5.py
import re
import requests
import time
import csv
from selenium import webdriver
from bs4 import BeautifulSoup
from requests.exceptions import RequestException

def generate_movie_list_link(i):
    movie_list_url = "https://www.imdb.com/search/title/?title_type=tv_movie&release_date=2019-01-01,2021-01-01&sort=num_votes,desc&start="+str((i-1)*50+1)+"&ref_=adv_nxt"
    return movie_list_url

def generate_movie_review_list_link(url):
    URL = url
    try:
        response = requests.get(URL)
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html,'lxml')
            movies = soup.select('.lister-item-content')
            #print(movies)
            movie_reviews_url_list = [[0] * 3 for j in range(50)]
            i = 0
            for movie in movies:
                header = movie.select_one('.lister-item-header')
                movie_link = header.select_one('a')['href']                     #电影链接
                id_pattern = re.compile(r'(?<=tt)\d+(?=/?)')
                movie_id = int(id_pattern.search(movie_link).group())           #imdb电影id
                movie_reviews_url = "https://www.imdb.com/title/tt"+str(movie_id)+"/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0"
                movie_name = header.select_one('a').string

                movie_reviews_url_list[i] = [movie_name, movie_id, movie_reviews_url]
                i += 1
                #print(i,movie_name,movie_id,movie_reviews_url)
                #time.sleep(1)
            return movie_reviews_url_list
        else:
            print("Error when request URL")
    except RequestException:
        print("Request Failed")
        return None

TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    text = str(text)
    clean_text = TAG_RE.sub('', text)
    clean_text = clean_text.replace("[", "")
    clean_text = clean_text.replace("]", "")
    return clean_text

#获取imdb电影评论页面所有非spoiler的用户评论
def get_imdb_movie_review(url,movieId,movieName):
    NETWORK_STATUS = True  # 判断状态变量
    URL = url
    print(URL)
    try:
        response = requests.get(URL)
        if response.status_code == 200:
            original_html = response.text
            original_soup = BeautifulSoup(original_html,'lxml')
            #需要下载Chromedriver
            driver = webdriver.Chrome("C:/Program Files/Google/Chrome/Application/chromedriver.exe")
            driver.get(URL)

            # 判断是否需要模拟点击load more 按钮
            while(1):
                temp_response = driver.page_source
                temp_html = temp_response
                temp_soup = BeautifulSoup(temp_html,'lxml')
                load_more = temp_soup.select('.ipl-load-more__button')
                #需要模拟点击
                if(len(load_more) == 1):
                    button_load_more = driver.find_element_by_class_name('ipl-load-more__button')
                    button_is_or_not_visible = driver.find_element_by_class_name('ipl-load-more__button').is_displayed()
                    if(button_is_or_not_visible is True):       #button按钮不可见时停止点击
                        button_load_more.click()
                        time.sleep(3)
                    else:
                        break
                else:
                    break

            final_response = driver.page_source
            html = final_response
            soup = BeautifulSoup(html,'lxml')

            movie_id = movieId
            reviews = soup.select('.review-container')
            movie_reviews_list = [[0] * 5 for j in range(1500)]
            movie_name = movieName

            i = 0
            for review in reviews:
                header = review.select_one('.display-name-date')
                user_link = header.select_one('a')['href']
                user_id_pattern = re.compile(r'(?<=ur)\d+(?=/?)')
                user_id = int(user_id_pattern.search(user_link).group())    #用户id
                review_date = header.select_one('.review-date').string      #用户评论时间

                content = review.select_one('.content')
                user_review = content.select('.text.show-more__control')     #用户评论
                user_review = remove_tags(user_review)

                movie_reviews_list[i][0] = user_id
                movie_reviews_list[i][1] = review_date
                movie_reviews_list[i][2] = user_review
                movie_reviews_list[i][3] = movie_id
                movie_reviews_list[i][4] = movie_name

                i += 1
                #print(i,user_id,review_date,user_review,movie_id)
                time.sleep(0.01)

            driver.close()      #关闭网页
            return movie_reviews_list
        else:
            print("Error when request URL")
    except RequestException:
        print("Request Failed")
        return None
    except requests.exceptions.Timeout:
        NETWORK_STATUS = False  # 请求超时改变状态
        if NETWORK_STATUS == False:
            #请求超时
            driver.close()  # 关闭网页
            print('请求超时，重复请求')
            get_imdb_movie_review(URL, movieId, movieName)

if __name__ == '__main__':
    i = 1
    countall = 0
    count = 0
    with open('movie_review_info.csv', 'w', newline="",encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile, dialect=("excel"))
        csvwriter.writerow(["userId", "reviewDate", "userReview","movieId","movieName"])
        #i表示页数，这里只爬取第一页
        while (i <= 1):
             print("第" + str(i) + "页电影")
             movie_list_url = generate_movie_list_link(i)
             movie_review_url_list = generate_movie_review_list_link(movie_list_url)
             j = 0

             while (j < 5):
                 print("第" + str((i-1) * 50 + j + 1) + "部电影的评论")
                 l = [[0] * 4 for j in range(1500)]
                 l = get_imdb_movie_review(movie_review_url_list[j][2], movie_review_url_list[j][1], movie_review_url_list[j][0])
                 k = 0
                 if(l is None):
                     j += 1
                     continue

                 while(l[k][0] != 0):
                     csvwriter.writerow(l[k])
                     k += 1
                     count += 1
                     countall += 1
                 print("已爬取第" + str((i-1) * 50 + j + 1) + "部电影的评论共" + str(count) + "条")
                 j += 1
                 count = 0
             i += 1

    print("爬虫完毕，共爬取"+str(countall)+"条用户评论")


第1页电影
第1部电影的评论
https://www.imdb.com/title/tt4943998/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0
已爬取第1部电影的评论共229条
第2部电影的评论
https://www.imdb.com/title/tt8425058/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0
已爬取第2部电影的评论共97条
第3部电影的评论
https://www.imdb.com/title/tt8022928/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0
已爬取第3部电影的评论共66条
第4部电影的评论
https://www.imdb.com/title/tt10090796/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0
已爬取第4部电影的评论共60条
第5部电影的评论
https://www.imdb.com/title/tt12885438/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0
已爬取第5部电影的评论共25条
爬虫完毕，共爬取477条用户评论
